In [64]:
#import libraries
import folium
import pandas as pd

In [65]:
#get the dataset
res_loc = pd.read_csv('res_location.csv',index_col='res')
u_del_cent = pd.read_csv('user_del_cent.csv',index_col='user_id')
u_res_cent = pd.read_csv('user_res_cent.csv',index_col='user_id')
u_del_res = pd.read_csv('user_del_res.csv',index_col='user_id')
u_rev_res = pd.read_csv('user_res_list.csv',index_col='user_id')

In [66]:
#remove those users who does not have delivery centroid
rm_user = []
for i in u_res_cent.index:
    if i not in u_del_cent.index:
        rm_user.append(i)
u_res_cent=u_res_cent.drop(rm_user)

In [67]:
#convert the location into lists
def modify(s):
    l=s.split(',')
    p=[round(float(x),4) for x in l[::2]]
    q=[round(float(x),4) for x in l[1::2]]
    r=[]
    for i in range(len(p)):
        r.append([p[i],q[i]])
    return r

In [68]:
for i in range(u_del_cent.shape[0]):
    u_del_cent.iloc[i]['centroid']=modify(u_del_cent.iloc[i]['centroid'])
for i in range(u_res_cent.shape[0]):
    u_res_cent.iloc[i]['centroid']=modify(u_res_cent.iloc[i]['centroid'])

In [69]:
#get the list of restaurants for plotting
for i in range(u_del_res.shape[0]):
    u_del_res.iloc[i]['res_list'] = u_del_res.iloc[i]['res_list'].split(',')
for i in range(u_rev_res.shape[0]):
    u_rev_res.iloc[i]['res_list'] = u_rev_res.iloc[i]['res_list'].split(',')

In [70]:
#get the data to plot user location based on delivery
cnt = 0
user_del = {}
for i in u_del_cent.index:
    res_list = []
    r_list = u_del_res.loc[i]['res_list']
    for r in r_list:
        f = {}
        if r in res_loc.index:
            f['name'] = r
            f['lat'] = round(res_loc.loc[r]['lat'],4)
            f['long'] = round(res_loc.loc[r]['long'],4)
            res_list.append(f)
    if len(res_list)>0:
        cnt += 1
        print(cnt,end = '\r')
        user_del[i] = pd.DataFrame(res_list)

In [71]:
#get the data to plot user location based on review
cnt = 0
user_rev = {}
for i in u_res_cent.index:
    res_list = []
    r_list = u_rev_res.loc[i]['res_list']
    for r in r_list:
        f = {}
        if r in res_loc.index:
            f['name'] = r
            f['lat'] = round(res_loc.loc[r]['lat'],4)
            f['long'] = round(res_loc.loc[r]['long'],4)
            res_list.append(f)
    if len(res_list)>0:
        cnt += 1 
        print(cnt,end = '\r')
        user_rev[i] = pd.DataFrame(res_list)

In [72]:
#plot the map using folium lib
def get_map(uid,u_del_cent,u_res_cent,user_del,user_rev):
    del_cen = u_del_cent.loc[uid]['centroid']
    res_cen = u_res_cent.loc[uid]['centroid']
    del_data = user_del[uid]
    rev_data = user_rev[uid]
    m=folium.Map(location=del_cen[0])
    fgrr=folium.FeatureGroup(name='Review Restaurants')
    for i in range(rev_data.shape[0]):
        folium.Marker([rev_data.iloc[i]['lat'], rev_data.iloc[i]['long']], popup=rev_data.iloc[i]['name'],icon=folium.Icon(color='blue', icon='info-sign')).add_to(fgrr)
    fgdr=folium.FeatureGroup(name='Delivery Restaurants')
    for i in range(del_data.shape[0]):
        folium.Marker([del_data.iloc[i]['lat'], del_data.iloc[i]['long']], popup=del_data.iloc[i]['name'],icon=folium.Icon(color='black', icon='info-sign')).add_to(fgdr)
    fgdl=folium.FeatureGroup(name='Delivery Locations')
    for i in range(len(del_cen)):
        folium.Marker(del_cen[i],popup='del addr:{}'.format(i+1),icon=folium.Icon(color='red', icon='remove-sign')).add_to(fgdl)
    fgrl=folium.FeatureGroup(name='Eating Locations')
    for i in range(len(res_cen)):
        folium.Marker(res_cen[i],popup='rev addr:{}'.format(i+1),icon=folium.Icon(color='purple', icon='remove-sign')).add_to(fgrl)
    m.add_child(fgrr)
    m.add_child(fgdr)
    m.add_child(fgrl)
    m.add_child(fgdl)
    m.add_child(folium.LayerControl())
    m.save('maps/{}.html'.format(uid))

In [75]:
#get the user ids and plot the maps
u_ids=u_del_cent.index
for i,uid in enumerate(u_ids):
    get_map(uid,u_del_cent,u_res_cent,user_del,user_rev)
    print(i,end='\r')